In [1]:
#0读取数据 并 处理该小问异常值
import pandas as pd
import numpy as np

cost = pd.read_csv('各年7.1-7.7单品批发价格.csv')
sale_category = pd.read_csv('各年7.1-7.7各品类销量.csv')
sale_product = pd.read_csv('各年7.1-7.7单品销量.csv')
price = pd.read_csv('各年7.1-7.7单品售价.csv')
price_discount = pd.read_csv('各年7.1-7.7单品折扣价格.csv')
p1 = pd.read_csv('p1.csv')
p2 = pd.read_csv('p2.csv')
c_df = pd.read_csv('Prophet单品批发价格预测.csv')

cost = cost.set_index(cost['日期'])
cost = cost.drop('日期', axis=1)

sale_category = sale_category.set_index(sale_category['销售日期'])
sale_category = sale_category.drop('销售日期', axis=1)

sale_product = sale_product.set_index(sale_product['销售日期'])
sale_product = sale_product.drop('销售日期', axis=1)

price = price.set_index(price['销售日期'])
price = price.drop('销售日期', axis=1)

price_discount = price_discount.set_index(price_discount['销售日期'])
price_discount = price_discount.drop('销售日期', axis=1)

c_df = c_df.set_index(c_df.iloc[:,0])
c_df = c_df.drop(c_df.columns[0], axis=1)

#处理cost中的异常值，作为基准处理price与sale_product中的异常值
num_costna = cost.isna().sum()
num_costna = num_costna[num_costna != 21]
cost_processed = cost[num_costna.index]
price_processed = price[cost_processed.columns]
# price_discount_processed = price_discount[cost_processed.columns]
c_df = c_df[cost_processed.columns]
sale_product_processed = sale_product[cost_processed.columns]

#处理sale_category中的异常值
product_info = pd.read_excel('附件1.xlsx')
info_dict = product_info[['单品名称', '分类名称']].set_index('单品名称').to_dict(orient='dict')['分类名称']

zero = np.zeros((21,6))
sale_category_processed =  pd.DataFrame(zero, columns=sale_category.columns, index=sale_category.index)
for col in sale_product_processed.columns:
    sale_category_processed[info_dict[col]] += sale_product_processed[col]

c_df

,七彩椒(1),七彩椒(2),上海青,东门口小白菜,云南油麦菜,云南生菜,云南生菜(份),余干椒,冰草,净藕(1),...,青杭椒(2),青梗散花,青线椒,青茄子(1),高瓜(1),鲜木耳(1),鲜木耳(份),黄心菜(1),黄心菜(2),黄白菜(2)
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
2023-07-01,11.283125,17.223246,4.448322,3.558951,4.288465,4.892639,3.156615,9.001972,9.141936,7.593090,...,6.633472,6.327193,8.030885,2.451122,10.695603,7.450570,1.269304,3.342631,3.572313,4.398471
2023-07-02,11.555013,17.286523,4.577368,3.703236,4.343757,4.982323,3.160196,11.219541,9.274606,7.332504,...,6.788234,6.232115,8.045755,2.528307,10.729535,7.467691,1.282147,3.227073,3.531505,4.456539
2023-07-03,11.877639,17.571826,4.558968,3.543832,4.274410,4.952225,3.096889,12.319600,9.393054,7.025901,...,6.831326,6.231063,7.845660,2.493652,10.709710,7.468844,1.273445,3.214212,3.444150,4.449769
2023-07-04,11.420698,18.210313,4.559555,3.516277,4.295227,4.932869,3.020461,13.788977,9.360426,6.748812,...,6.916969,6.223693,7.936348,2.479481,10.753400,7.424200,1.285398,3.169600,3.391892,4.500709
2023-07-05,11.405392,18.708175,4.642817,3.744003,4.316992,4.958944,2.892764,14.328330,9.473411,6.433894,...,6.926025,6.105857,8.017064,2.529456,10.939844,7.369914,1.300645,3.174790,3.274633,4.473733
2023-07-06,11.631606,19.020030,4.736267,3.683155,4.248266,4.927594,2.929317,14.765293,9.323351,6.053643,...,7.135817,6.126682,7.856343,2.530739,10.909184,7.362216,1.299923,3.133978,3.299617,4.509486
2023-07-07,12.235706,19.603635,4.667127,3.635333,4.305486,4.672407,2.788980,13.550692,9.308037,5.701222,...,7.045419,6.119803,8.271865,2.452341,10.770628,7.423415,1.301105,3.050977,3.204841,4.465784


In [2]:
#1.计算S_gt

S_max = sale_category_processed.apply(lambda x:x.max())
S_min = sale_category_processed.apply(lambda x:x.min())

S_gt = (sale_category_processed - S_min) / (S_max - S_min)
S_gt

,水生根茎类,花叶类,花菜类,茄类,辣椒类,食用菌
销售日期,,,,,,
2020-07-01,0.035020,0.649325,0.534132,0.548335,0.706815,0.438047
2020-07-02,0.030933,0.545266,0.458425,0.482229,0.560319,0.799339
2020-07-03,0.027826,0.593928,0.413310,0.559204,0.535410,0.634997
2020-07-04,0.044650,1.000000,0.741609,1.000000,0.763524,0.753677
2020-07-05,0.021434,0.877858,0.737066,0.871932,1.000000,0.926712
2020-07-06,0.000000,0.581589,0.418167,0.340544,0.458786,0.476180
2020-07-07,0.006687,0.388773,0.502066,0.480418,0.232267,0.788750
2021-07-01,0.094891,0.296490,0.211391,0.358950,0.402187,0.224630
2021-07-02,0.132134,0.169250,0.000000,0.406567,0.359975,0.277632


In [3]:
#2.计算w
import datetime
w_k = price_processed / cost_processed - 1
# 0不重要，对应的当日单品销量会为0
w_k = w_k.fillna(0)
w_prime = w_k * sale_product_processed

zero = np.zeros((21,6))
w_x_Θ =  pd.DataFrame(zero, columns=sale_category.columns, index=sale_category.index)
for col in w_prime.columns:
    w_x_Θ[info_dict[col]] += w_prime[col]

w = w_x_Θ / sale_category_processed

t_list = []
for date in w.index:
    time = datetime.datetime.strptime(date, '%Y-%m-%d')
    t_list.append(time.day)
t_np = np.array(t_list)
t_norm = (t_np - t_np.min()) / (t_np.max() - t_np.min())
w['date_factor'] = t_norm
w

,水生根茎类,花叶类,花菜类,茄类,辣椒类,食用菌,date_factor
销售日期,,,,,,,
2020-07-01,0.706978,0.590086,0.510566,0.685442,0.820073,0.576699,0.000000
2020-07-02,0.710128,0.757272,0.591105,1.355157,0.744661,0.650298,0.166667
2020-07-03,0.782531,0.699971,0.598779,0.902886,0.623897,0.729150,0.333333
2020-07-04,0.688605,0.648614,0.553701,0.879172,0.737760,0.685781,0.500000
2020-07-05,0.845018,0.670352,0.512468,0.799566,0.670179,0.675019,0.666667
2020-07-06,0.834862,0.641013,0.533176,0.891419,0.729580,0.673781,0.833333
2020-07-07,0.831502,0.667985,0.590915,0.829822,0.834436,0.728714,1.000000
2021-07-01,0.683266,0.754930,0.548351,0.814191,0.880824,0.621025,0.000000
2021-07-02,0.719280,0.776297,0.701186,0.758584,1.059710,0.617144,0.166667


In [4]:
#3.模型拟合预测S_j
import numpy as np
import math
class MyLogisticRegression:
    def __init__(self):
        """
        Class contructor.
        
        :attr beta: weights vector of logistic regression.
        """
        self.beta = None

    def fx(self, X):
        """
        Calculate the value of f(x) given x.

        :param X: numpy.ndarray with a shape of (n, d), input data.
        :return:
            fx_value: numpy.ndarray with a length of n, output of f(x).
        """
        n = X.shape[0]
        fx_value = np.zeros(n)
        for i in range(n):
            data = self.beta@X[i].T
            fx_value[i] = data
        fx_value = 1/(1+np.exp(-fx_value))
        return fx_value

    def loss(self, fx_value, y):
        """
        Calculate the loss function given the calculated value f(x) and the true label y.

        :param fx_value: numpy.ndarray with a length of n, 
                         a vector of hypothesis function values on these samples, which is the output of the function fx
        :param y: numpy.ndarray with a length of n, 
                  a vector of the true labels of these samples
        :return:
            CELoss: a float value of the cross-entropy loss.
        """
        loss = 0
        n = fx_value.shape[0]
        for i in range(n):
            loss = loss + (-y[i])*math.log(fx_value[i])-(1-y[i])*math.log((1-fx_value[i]))
        loss = loss / n
        return loss

    def fit(self, X, y, n_iters = 10000, alpha = 0.01):
        """
        Train the model using gradient descent

        :param X: numpy.ndarray with a shape of (n*d), input data.
        :param y: numpy.ndarray with a length of n, the true labels of these samples
        :param n_iters: int, number of iterations
        :param alpha: float, learning rate
        :return:
            CELoss_list: list with a length of n_iters+1, 
                         contains the loss values before the gradient descent and after the gradient descent.
        """

        n, d = X.shape
        
        self.beta = np.zeros(d+1)
        #the first element in X_ is 1
        X_ = np.column_stack([np.ones(n), X])
        CELoss_list = [self.loss(self.fx(X_), y)]
        
        for i in range(n_iters):
            fx_value = self.fx(X_)
            offset = np.zeros(d+1)
            for j in range(n):
                offset = offset + (-y[j]/fx_value[j]+(1-y[j])/(1-fx_value[j]))*fx_value[j]*(1-fx_value[j])*X_[j]
            offset = offset / n
            self.beta = self.beta - offset #*alpha
            
            CELoss_list.append(self.loss(self.fx(X_), y))
        
        return CELoss_list

    def predict(self, X):
        """
        Predict the labels of input instances.

        :param X: numpy.ndarray with a shape of (n*d), input data.
        :return:
            y_hat: numpy.ndarray with a length of n, the predicted labels of these samples
        """
        n = X.shape[0]
        X = np.column_stack([np.ones(n), X])
        fx_value = self.fx(X)
        return fx_value

In [6]:
#得到七天各自S预测模型
S_model = {}
for i in S_gt.columns:
    X = w[[i, 'date_factor']].values
    y = S_gt[i].values
    my_model = MyLogisticRegression()
    my_model.fit(X, y)
    S_model[i] = my_model

    x = np.zeros((1,2))
    x[0,0] = 1.2
    x[0,1] = 0.0
    S_model[i].predict(x)
    print(i, my_model.beta, S_model[i].predict(x))


水生根茎类 [ 0.87044213 -3.07242384  0.31215144] [0.05644082]
花叶类 [ 5.49034117 -7.43220486 -0.38073743] [0.03142249]
花菜类 [-0.34783207 -0.37234675  0.73931001] [0.31117149]
茄类 [ 0.62168268 -0.74610861 -0.08747892] [0.43201182]
辣椒类 [-0.55381326  0.79281676 -0.56631668] [0.59810293]
食用菌 [-5.83608669  9.24336588 -0.17446366] [0.99481069]


In [9]:
import pandas as pd
import numpy as np

price_product = pd.read_csv('按单品每天售价.csv')
c_df = pd.read_csv('Prophet单品批发价格预测.csv')
sale_category = pd.read_csv('6.24-6.30各品类销量.csv')
sale_product = pd.read_csv('6.24-6.30单品总销量.csv')
sale_discount_product = pd.read_csv('6.24-6.30单品未折销量.csv')
cost = pd.read_csv('6.24-6.30单品批发价格.csv')
price_product_discount = pd.read_csv('6.24-6.30单品折扣价格.csv')

p1_7 = pd.read_csv('p1_7.csv')
p2_7 = pd.read_csv('p2_7.csv')

cost = cost.set_index(cost['日期'])
cost = cost.drop('日期', axis=1)

price_product = price_product.set_index(price_product['销售日期'])
price_product = price_product.drop('销售日期', axis=1)

price_product_discount = price_product_discount.set_index(price_product_discount['销售日期'])
price_product_discount = price_product_discount.drop('销售日期', axis=1)


sale_product = sale_product.set_index(sale_product['销售日期'])
sale_product = sale_product.drop('销售日期', axis=1)

sale_discount_product = sale_discount_product.set_index(sale_discount_product['销售日期'])
sale_discount_product = sale_discount_product.drop('销售日期', axis=1)

sale_category = sale_category.set_index(sale_category['销售日期'])
sale_category = sale_category.drop('销售日期', axis=1)

price_product = price_product.loc[['2023-06-24', '2023-06-25', '2023-06-26', '2023-06-27', '2023-06-28', '2023-06-29', '2023-06-30']]

num_na = price_product.isna().sum()
num_na = num_na[num_na == len(price_product)]
# price_product = price_product[]
num_costna = cost.isna().sum()
num_costna = num_costna[num_costna != 7]
cost_processed = cost[num_costna.index]

#基准1
price_product_processed = price_product.drop(num_na.index, axis=1)
cost_processed = cost_processed[price_product_processed.columns]

sale_product_processed = sale_product[cost_processed.columns]
sale__discount_product_processed = sale_discount_product[cost_processed.columns]

#处理sale_category中的异常值
product_info = pd.read_excel('附件1.xlsx')
info_dict = product_info[['单品名称', '分类名称']].set_index('单品名称').to_dict(orient='dict')['分类名称']


zero = np.zeros((7,6))
sale_category_processed =  pd.DataFrame(zero, columns=sale_category.columns, index=sale_category.index)
for col in sale_product_processed.columns:
    sale_category_processed[info_dict[col]] += sale_product_processed[col]


price_product_processed

,七彩椒(2),上海青,云南油麦菜,云南油麦菜(份),云南生菜,云南生菜(份),净藕(1),双孢菇(盒),圆茄子(2),外地茼蒿,...,西兰花,西峡花菇(1),野生粉藕,金针菇(盒),长线茄,青红杭椒组合装(份),青线椒(份),青茄子(1),高瓜(1),高瓜(2)
销售日期,,,,,,,,,,,,,,,,,,,,,
2023-06-24,19.8,8.0,NaN,4.5,NaN,4.5,16.0,5.5,8.0,12.0,...,14.0,24.0,NaN,2.0,12.0,5.8,4.8,6.0,16.0,NaN
2023-06-25,20.8,8.0,7.2,4.5,9.2,4.5,16.0,5.5,8.0,NaN,...,14.0,24.0,NaN,2.0,12.0,5.8,NaN,6.0,16.0,18.0
2023-06-26,20.8,8.0,7.2,4.5,9.2,4.5,16.0,5.5,NaN,13.2,...,14.0,24.0,NaN,2.0,12.0,NaN,3.8,6.0,16.0,18.0
2023-06-27,20.8,8.0,7.2,4.5,9.2,4.5,16.0,5.5,8.0,13.2,...,12.0,24.0,26.0,2.0,12.0,5.8,NaN,6.0,16.0,NaN
2023-06-28,20.8,8.0,NaN,4.5,NaN,4.5,16.0,5.5,5.2,NaN,...,12.0,24.0,NaN,2.0,12.0,5.8,NaN,6.0,16.0,NaN
2023-06-29,20.8,8.0,NaN,4.5,9.2,4.5,16.0,5.5,5.2,NaN,...,11.2,24.0,26.0,2.0,12.0,5.8,NaN,6.0,16.0,18.0
2023-06-30,NaN,8.0,NaN,4.5,NaN,5.8,16.0,5.5,5.2,NaN,...,11.2,24.0,26.0,2.0,12.0,5.8,NaN,NaN,16.0,NaN


In [27]:
#1.计算S_gt
S_max = sale_category_processed.apply(lambda x:x.max())
S_min = sale_category_processed.apply(lambda x:x.min())

S_gt = (sale_category_processed - S_min) / (S_max - S_min)
S_gt

,水生根茎类,花叶类,花菜类,茄类,辣椒类,食用菌
销售日期,,,,,,
2023-06-24,0.451397,1.000000,0.173215,1.000000,1.000000,1.000000
2023-06-25,0.486904,0.350385,0.129874,0.746717,0.093767,0.000000
2023-06-26,0.000000,0.000000,0.000000,0.641926,0.187128,0.016447
2023-06-27,0.483242,0.577417,0.309388,0.000000,0.000000,0.181765
2023-06-28,0.363188,0.613281,0.399220,0.362634,0.052241,0.976836
2023-06-29,1.000000,0.673896,0.814037,0.155157,0.812539,0.689777
2023-06-30,0.719290,0.616764,1.000000,0.807607,0.560313,0.227458


In [28]:
#2.计算w
import datetime
w_k = price_product_processed / cost_processed - 1
# 0不重要，对应的当日单品销量会为0
w_k = w_k.fillna(0)
w_prime = w_k * sale_product_processed

zero = np.zeros((7,6))
w_x_Θ =  pd.DataFrame(zero, columns=sale_category.columns, index=sale_category.index)
for col in w_prime.columns:
    w_x_Θ[info_dict[col]] += w_prime[col]

w = w_x_Θ / sale_category_processed

t_list = []
for date in w.index:
    time = datetime.datetime.strptime(date, '%Y-%m-%d')
    t_list.append(time.day)
t_np = np.array(t_list)
t_norm = (t_np - t_np.min()) / (t_np.max() - t_np.min())
w['date_factor'] = t_norm
w

,水生根茎类,花叶类,花菜类,茄类,辣椒类,食用菌,date_factor
销售日期,,,,,,,
2023-06-24,0.465825,0.538826,0.616461,0.581610,0.857332,0.509061,0.000000
2023-06-25,0.446359,0.493291,0.717791,0.581124,0.965484,0.487209,0.166667
2023-06-26,0.434034,0.687008,0.763224,0.567975,1.011998,0.539387,0.333333
2023-06-27,0.460455,0.564495,0.538462,0.654855,1.060590,0.501020,0.500000
2023-06-28,0.477489,0.504028,0.577152,0.588295,1.071339,0.476035,0.666667
2023-06-29,0.467426,0.515042,0.514073,0.632503,0.924950,0.452413,0.833333
2023-06-30,0.478296,0.672129,0.505035,0.729765,0.858868,0.458399,1.000000


In [29]:
#3.处理得到avg_discount
discount_temp1 = price_product_discount / price_product_processed
missing_col = [col for col in price_product_processed.columns if col not in price_product_discount.columns]
discount_temp1[missing_col] = 1
discount_k = discount_temp1[price_product_processed.columns].fillna(1)

sale_diff = (sale_product_processed - sale__discount_product_processed)
discount_prime = discount_k * sale_diff

discount_x_Θ = discount_prime.apply(lambda x: x.sum())
discount_sum = sale_diff.apply(lambda x:x.sum())

discount_df = discount_x_Θ / discount_sum
discount = discount_df.fillna(1)
discount

七彩椒(2)          0.600967
上海青             1.000000
云南油麦菜           1.000000
云南油麦菜(份)        0.623492
云南生菜            1.000000
云南生菜(份)         0.625287
净藕(1)           0.500000
双孢菇(盒)          0.565775
圆茄子(2)          1.000000
外地茼蒿            1.000000
奶白菜             1.000000
姜蒜小米椒组合装(小份)    0.762500
娃娃菜             0.639037
小皱皮(份)          0.584821
小米椒(份)          0.603448
小青菜(1)          1.000000
木耳菜             0.566994
木耳菜(份)          1.000000
枝江青梗散花          0.521462
洪湖藕带            0.500000
海鲜菇(包)          0.600000
白玉菇(袋)          1.000000
竹叶菜             1.000000
紫茄子(1)          1.000000
紫茄子(2)          1.000000
红椒(2)           0.600000
红莲藕带            1.000000
红薯尖             1.000000
芜湖青椒(1)         1.000000
苋菜              1.000000
菜心              1.000000
菠菜              1.000000
菠菜(份)           0.538603
菱角              1.000000
虫草花(份)          1.000000
螺丝椒             1.000000
螺丝椒(份)          0.593220
蟹味菇与白玉菇双拼(盒)    0.801724
西兰花             1.000000
西峡花菇(1)         1.000000


In [38]:
#4.生成A 和 B
sum_1 = sale_product_processed.apply(lambda x:x.sum())
sum_2 = sale__discount_product_processed.apply(lambda x:x.sum())
A = sum_2 / sum_1
B = 1 - A

七彩椒(2)          0.768694
上海青             1.000000
云南油麦菜           1.000000
云南油麦菜(份)        0.765101
云南生菜            1.000000
云南生菜(份)         0.827434
净藕(1)           0.693415
双孢菇(盒)          0.757143
圆茄子(2)          1.000000
外地茼蒿            1.000000
奶白菜             1.000000
姜蒜小米椒组合装(小份)    0.897959
娃娃菜             0.849315
小皱皮(份)          0.797468
小米椒(份)          0.980000
小青菜(1)          1.000000
木耳菜             0.713536
木耳菜(份)          0.333333
枝江青梗散花          0.797564
洪湖藕带            0.516860
海鲜菇(包)          0.741935
白玉菇(袋)          0.500000
竹叶菜             1.000000
紫茄子(1)          1.000000
紫茄子(2)          1.000000
红椒(2)           0.815439
红莲藕带            1.000000
红薯尖             1.000000
芜湖青椒(1)         1.000000
苋菜              1.000000
菜心              1.000000
菠菜              1.000000
菠菜(份)           0.510204
菱角              1.000000
虫草花(份)          0.875000
螺丝椒             1.000000
螺丝椒(份)          0.962025
蟹味菇与白玉菇双拼(盒)    0.500000
西兰花             1.000000
西峡花菇(1)         1.000000


In [31]:
#5.处理制作p1_7,p2_7的字典
p1_7_dict = p1_7.set_index('分类名称').to_dict(orient='dict')['0']
p2_7_dict = p2_7.set_index('分类名称').to_dict(orient='dict')['0']
p1_7_dict

{'水生根茎类': 0.6474439169851416,
 '花叶类': 0.8593728545143307,
 '花菜类': 0.9548922701512726,
 '茄类': 1.0,
 '辣椒类': 0.93864842022306,
 '食用菌': 0.7963596869757474}

In [32]:
#6.计算得到c
c = c_df[price_product_processed.columns]
c

,七彩椒(2),上海青,云南油麦菜,云南油麦菜(份),云南生菜,云南生菜(份),净藕(1),双孢菇(盒),圆茄子(2),外地茼蒿,...,西兰花,西峡花菇(1),野生粉藕,金针菇(盒),长线茄,青红杭椒组合装(份),青线椒(份),青茄子(1),高瓜(1),高瓜(2)
0,17.223246,4.448322,4.288465,2.855946,4.892639,3.156615,7.593090,3.219108,3.839041,8.175827,...,9.134922,12.556894,16.191241,1.352606,6.442921,3.264275,2.019804,2.451122,10.695603,13.132365
1,17.286523,4.577368,4.343757,2.875918,4.982323,3.160196,7.332504,3.191729,3.756583,8.089711,...,9.169098,11.816055,16.299760,1.319728,6.200920,3.279441,1.815960,2.528307,10.729535,13.497809
2,17.571826,4.558968,4.274410,2.874551,4.952225,3.096889,7.025901,3.117687,3.794525,7.932610,...,9.203346,10.975354,16.171141,1.291699,6.109356,3.318258,1.751134,2.493652,10.709710,13.708210
3,18.210313,4.559555,4.295227,2.823719,4.932869,3.020461,6.748812,3.087155,3.742297,8.215387,...,9.193243,10.255791,14.626073,1.265136,5.931516,3.381411,1.470723,2.479481,10.753400,13.333739
4,18.708175,4.642817,4.316992,2.768497,4.958944,2.892764,6.433894,3.028007,3.740992,8.394069,...,9.215955,9.532399,12.407567,1.333062,5.907775,3.443833,1.703397,2.529456,10.939844,13.773311
5,19.020030,4.736267,4.248266,2.792756,4.927594,2.929317,6.053643,2.939994,3.738289,9.095167,...,9.215753,8.840199,9.243621,1.346646,5.706416,3.434658,1.494808,2.530739,10.909184,13.943592
6,19.603635,4.667127,4.305486,2.746508,4.672407,2.788980,5.701222,2.933561,3.684557,8.908615,...,9.241625,7.700362,4.716894,1.351109,5.562453,3.456292,1.305437,2.452341,10.770628,13.971548


In [33]:
#7.读取yita
l = pd.read_excel('附件4.xlsx', sheet_name='Sheet1')
l.drop('单品编码', axis=1)
l['损耗率(%)'] = l['损耗率(%)'] / 100
loss_rate = l.set_index('单品名称').to_dict(orient= 'dict')['损耗率(%)']
loss_rate

{'牛首生菜': 0.043899999999999995,
 '四川红香椿': 0.10460000000000001,
 '西峡花菇(1)': 0.10800000000000001,
 '本地小毛白菜': 0.0018,
 '白菜苔': 0.08779999999999999,
 '苋菜': 0.1852,
 '云南生菜': 0.1525,
 '竹叶菜': 0.1362,
 '小白菜': 0.0759,
 '南瓜尖': 0.1346,
 '上海青': 0.14429999999999998,
 '萝卜叶': 0.0535,
 '牛首油菜': 0.1217,
 '茼蒿': 0.06269999999999999,
 '蔡甸藜蒿': 0.004699999999999999,
 '菜心': 0.13699999999999998,
 '木耳菜': 0.0761,
 '大白菜': 0.2227,
 '豌豆尖': 0.0836,
 '云南油麦菜': 0.1281,
 '藕': 0.1532,
 '红尖椒': 0.0673,
 '青尖椒': 0.0672,
 '红椒(1)': 0.1176,
 '紫茄子(2)': 0.060700000000000004,
 '青茄子(1)': 0.0501,
 '西峡香菇(1)': 0.1382,
 '金针菇(1)': 0.034300000000000004,
 '马齿苋': 0.11779999999999999,
 '西兰花': 0.0926,
 '本地菠菜': 0.10460000000000001,
 '黄心菜(1)': 0.10640000000000001,
 '黑油菜': 0.001,
 '平菇': 0.11599999999999999,
 '净藕(1)': 0.0554,
 '青杭椒(1)': 0.0,
 '姬菇(1)': 0.08990000000000001,
 '红杭椒': 0.0999,
 '泡泡椒(精品)': 0.0708,
 '七彩椒(1)': 0.1598,
 '白玉菇(1)': 0.0943,
 '本地上海青': 0.1199,
 '菠菜': 0.18510000000000001,
 '高瓜(1)': 0.2925,
 '娃娃菜': 0.0248,
 '双孢菇': 0.1278,
 '蟹味菇(1)

In [34]:
result = pd.DataFrame(index= [0], columns= sale_product_processed.columns)
result

,七彩椒(2),上海青,云南油麦菜,云南油麦菜(份),云南生菜,云南生菜(份),净藕(1),双孢菇(盒),圆茄子(2),外地茼蒿,...,西兰花,西峡花菇(1),野生粉藕,金针菇(盒),长线茄,青红杭椒组合装(份),青线椒(份),青茄子(1),高瓜(1),高瓜(2)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
product_info = pd.read_excel('附件1.xlsx')
info_dict = product_info[['单品名称', '分类名称']].set_index('单品名称').to_dict(orient='dict')['分类名称']

{'牛首生菜': '花叶类',
 '四川红香椿': '花叶类',
 '本地小毛白菜': '花叶类',
 '白菜苔': '花叶类',
 '苋菜': '花叶类',
 '云南生菜': '花叶类',
 '竹叶菜': '花叶类',
 '小白菜': '花叶类',
 '南瓜尖': '花叶类',
 '上海青': '花叶类',
 '萝卜叶': '花叶类',
 '牛首油菜': '花叶类',
 '茼蒿': '花叶类',
 '蔡甸藜蒿': '花叶类',
 '菜心': '花叶类',
 '木耳菜': '花叶类',
 '大白菜': '花叶类',
 '豌豆尖': '花叶类',
 '云南油麦菜': '花叶类',
 '马齿苋': '花叶类',
 '本地菠菜': '花叶类',
 '黄心菜(1)': '花叶类',
 '黑油菜': '花叶类',
 '本地上海青': '花叶类',
 '菠菜': '花叶类',
 '娃娃菜': '花叶类',
 '红薯尖': '花叶类',
 '枝江红菜苔': '花叶类',
 '黄花菜': '花叶类',
 '快菜': '花叶类',
 '随州泡泡青': '花叶类',
 '田七': '花叶类',
 '东门口小白菜': '花叶类',
 '外地茼蒿': '花叶类',
 '冰草': '花叶类',
 '紫苏': '花叶类',
 '薄荷叶': '花叶类',
 '蒲公英': '花叶类',
 '丝瓜尖': '花叶类',
 '奶白菜': '花叶类',
 '芥菜': '花叶类',
 '大芥兰': '花叶类',
 '面条菜': '花叶类',
 '甜白菜': '花叶类',
 '荠菜': '花叶类',
 '马兰头': '花叶类',
 '甘蓝叶': '花叶类',
 '洪山菜苔': '花叶类',
 '本地黄心油菜': '花叶类',
 '青菜苔': '花叶类',
 '小青菜(1)': '花叶类',
 '鲜粽子叶': '花叶类',
 '艾蒿': '花叶类',
 '奶白菜苗': '花叶类',
 '菊花油菜': '花叶类',
 '双沟白菜': '花叶类',
 '枝江红菜苔(份)': '花叶类',
 '云南生菜(份)': '花叶类',
 '云南油麦菜(份)': '花叶类',
 '茼蒿(份)': '花叶类',
 '菠菜(份)': '花叶类',
 '菜心(份)': '花叶类',
 '上海青(份)': '花叶类',
 '小青菜(2

In [47]:
result = pd.DataFrame(index= [0], columns=price_product_processed.columns)
result

,七彩椒(2),上海青,云南油麦菜,云南油麦菜(份),云南生菜,云南生菜(份),净藕(1),双孢菇(盒),圆茄子(2),外地茼蒿,...,西兰花,西峡花菇(1),野生粉藕,金针菇(盒),长线茄,青红杭椒组合装(份),青线椒(份),青茄子(1),高瓜(1),高瓜(2)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
#结果生成
import numpy as np
import random

#用于后续筛选
result_list = []

label_list = result.columns 
day = 0

for label in label_list:
    # 设置参数：温度、最低温度、降温系数、解空间
    f_list = []
    x_dict = {}
    S_dict = {}
    for iter in range(10):
        T = 20000
        Tmin = 1e-4
        delta = 0.95
        bound = [0, 1.5]
        # 定义目标函数
        func = lambda w: (p1_7_dict[info_dict[label]] * (S_model[info_dict[label]].predict(np.array([w, day/6]).reshape(1,2))[0] * (S_max[info_dict[label]] - S_min[info_dict[label]]) + S_min[info_dict[label]]) * (1 + w) * c[label][day] * A[label]+
                        p2_7_dict[info_dict[label]] * (S_model[info_dict[label]].predict(np.array([w, day/6]).reshape(1,2))[0] * (S_max[info_dict[label]] - S_min[info_dict[label]]) + S_min[info_dict[label]]) * (1 + w) * c[label][day] * discount[label] * B[label] -
                        (S_model[info_dict[label]].predict(np.array([w, day/6]).reshape(1,2))[0] * (S_max[info_dict[label]] - S_min[info_dict[label]]) + S_min[info_dict[label]]) * c[label][day] / (1 - loss_rate[label])) * (p1_7_dict[info_dict[label]]*A[label] + p2_7_dict[info_dict[label]]*B[label])
        # 初始值定义（也表示最优值）
        x = random.uniform(bound[0], bound[1])
        f = func(x)

        # epoch = 0
        while T > Tmin:
            # 产生新解及新函数值
            x_new = x + (random.random() * 2 - 1) * T
            while (not 0 < x_new < 1.5) or   (S_model[info_dict[label]].predict(np.array([x_new, day/6]).reshape(1,2))[0] * (S_max[info_dict[label]] - S_min[info_dict[label]]) + S_min[info_dict[label]]) * (p1_7_dict[info_dict[label]]*A[label] + p2_7_dict[info_dict[label]]*B[label]) < 2.5:# 保证生成的解在解空间中，使得每次温度降低是有意义的
                x_new = x + (random.random() * 2 - 1) * T
            f_new = func(x_new)
            # 计算增量
            if f_new > f:  # 以1的概率接受新解
                x, f = x_new, f_new
            else:
                if pow(np.e, (f - f_new) / T) > random.random():  # 根据Metropolis准则接受新解
                    x, f = x_new, f_new
            T *= delta
        f_list.append(f)
        x_dict[f] = x
        S_dict[f] = (S_model[info_dict[label]].predict(np.array([x, day/6]).reshape(1,2))[0] * (S_max[info_dict[label]] - S_min[info_dict[label]]) + S_min[info_dict[label]])/ (1 - loss_rate[label])
            # epoch += 1
        # print(f'函数的最大值为：{f},此时变量的值为：{x}')
    f_max = np.array(f_list).max()
    x_max = x_dict[f_max]
    S_j_max = S_dict[f_max]
    result.loc[day, label] = (f_max, x_max, S_j_max)
    result_list.append((f_max, label))

C:\Users\谢嘉楠\AppData\Local\Temp\ipykernel_1864\1723088451.py:40: RuntimeWarning: overflow encountered in double_scalars
  if pow(np.e, (f - f_new) / T) > random.random():  # 根据Metropolis准则接受新解


In [51]:
#保存问题二结果
result.to_excel('问题三加成参数与补货量最优方案(未筛选).xlsx')

In [53]:
#在上面结果中挑选27-33个最盈利的单品
result_sorted = sorted(result_list, reverse=True)
#筛选出盈利单品
product_chosen = []
for i in result_sorted:
    if i[0] > 0:
        product_chosen.append(i[1])

if len(product_chosen) > 33:
    product_chosen[0:33]
elif len(product_chosen) < 27:
    pass #若此情况，则回头重新运行模拟退火单元格，寻找新的最大利润

#问题三最终方案
result[product_chosen].to_excel('问题三加成参数与补货量最优方案(筛选).xlsx')
